In [27]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
wordnet_lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [29]:
df = pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [30]:
def normalizer(tweets):
      only_letters = re.sub("[^a-zA-Z]", " ", tweets)
      only_letters = only_letters.lower()
      only_letters = only_letters.split()
      filtered_result = [word for word in only_letters if word not in stopwords.words('english')]
      lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
      lemmas = ' '.join(lemmas)
      return lemmas

In [31]:
df = shuffle(df)
y = df['airline_sentiment']
x = df.text.apply(normalizer)

In [78]:
vectorizer = CountVectorizer()
x_vectorized = vectorizer.fit_transform(x)


['tfidf_vectorizer.pkl']

In [100]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_vectorized.shape[1],)),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

In [101]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [102]:
train_x, val_x, train_y, val_y = train_test_split(x_vectorized, y, test_size=0.2, random_state=42)

In [106]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels
train_y_encoded = label_encoder.fit_transform(y)

print("Encoded labels:", train_y_encoded)

Encoded labels: [0 0 1 ... 0 0 0]


In [108]:
print("train_x type:", type(train_x))
print("train_y_encoded type:", type(train_y_encoded))
print("train_x shape:", train_x.shape)
print("train_y_encoded shape:", train_y_encoded.shape)


train_x type: <class 'scipy.sparse._csr.csr_matrix'>
train_y_encoded type: <class 'numpy.ndarray'>
train_x shape: (11712, 12572)
train_y_encoded shape: (14640,)


In [112]:
# Convert train_x and val_x to dense numpy arrays
train_x_dense = train_x.toarray()
val_x_dense = val_x.toarray()

# Ensure train_y and val_y are properly encoded
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

# Train the model
history = model.fit(train_x_dense, train_y_encoded, epochs=10, batch_size=32, validation_data=(val_x_dense, val_y_encoded))


Epoch 1/10
366/366 [==============================] - 13s 31ms/step - loss: 0.6284 - accuracy: 0.7419 - val_loss: 0.5378 - val_accuracy: 0.7787
Epoch 2/10
366/366 [==============================] - 7s 20ms/step - loss: 0.3434 - accuracy: 0.8695 - val_loss: 0.5774 - val_accuracy: 0.7698
Epoch 3/10
366/366 [==============================] - 8s 21ms/step - loss: 0.1998 - accuracy: 0.9273 - val_loss: 0.6654 - val_accuracy: 0.7671
Epoch 4/10
366/366 [==============================] - 7s 20ms/step - loss: 0.1188 - accuracy: 0.9578 - val_loss: 0.8270 - val_accuracy: 0.7531
Epoch 5/10
366/366 [==============================] - 7s 19ms/step - loss: 0.0744 - accuracy: 0.9742 - val_loss: 0.9642 - val_accuracy: 0.7657
Epoch 6/10
366/366 [==============================] - 8s 22ms/step - loss: 0.0478 - accuracy: 0.9851 - val_loss: 1.0782 - val_accuracy: 0.7497
Epoch 7/10
366/366 [==============================] - 7s 19ms/step - loss: 0.0329 - accuracy: 0.9890 - val_loss: 1.2338 - val_accuracy: 0.755

In [116]:
import numpy as np

# Make predictions on the entire dataset
predicted_probabilities = model.predict_proba(data_features)

# Extract the index of the highest probability for each sample
predicted_labels = np.argmax(predicted_probabilities, axis=1)

# Convert predicted labels to sentiment categories
sentiment_categories = ['negative', 'neutral', 'positive']
predicted_sentiments = [sentiment_categories[label] for label in predicted_labels]

# Add predicted sentiment and scores to the DataFrame
data['predicted_sentiment'] = predicted_sentiments
data['negative_score'] = predicted_probabilities[:, 0]
data['neutral_score'] = predicted_probabilities[:, 1]
data['positive_score'] = predicted_probabilities[:, 2]

# Save the DataFrame with predictions and sentiment scores if needed
data.to_csv('data_with_predictions_and_scores.csv', index=False)


InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[3] = [0,5515] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse] name: 

In [80]:
regressor = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = regressor.fit(train_x, train_y)
joblib.dump(model, 'logistic_regression.pkl')

['logistic_regression.pkl']

In [35]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
gs_clf = GridSearchCV(model, params, n_jobs=1, cv=5)
gs_clf = gs_clf.fit(train_x, train_y)
model = gs_clf.best_estimator_

In [36]:
y_pred = model.predict(val_x)

_f1 = f1_score(val_y, y_pred, average='micro')
_confusion = confusion_matrix(val_y, y_pred)
__precision = precision_score(val_y, y_pred, average='micro')
_recall = recall_score(val_y, y_pred, average='micro')
_statistics = {'f1_score': _f1,
               'confusion_matrix': _confusion,
               'precision': __precision,
               'recall': _recall
              }

In [37]:
print(_statistics)

{'f1_score': 0.7795081967213114, 'confusion_matrix': array([[1985,  210,   81],
       [ 255,  458,   72],
       [ 102,   87,  410]], dtype=int64), 'precision': 0.7795081967213114, 'recall': 0.7795081967213114}


In [123]:
# Transform the text using the vectorizer
test_feature = vectorizer.transform(['Meat Week Day 3: Tummy hurts every night'])

# Predict the sentiment
predicted_probabilities = model.predict(test_feature)

# Extract the index of the highest probability
predicted_label = np.argmax(predicted_probabilities)

# Convert predicted label to sentiment category
sentiment_categories = ['negative', 'neutral', 'positive']
predicted_sentiment = sentiment_categories[predicted_label]

# Get the probability for each sentiment
negative_score = predicted_probabilities[0][0]
neutral_score = predicted_probabilities[0][1]
positive_score = predicted_probabilities[0][2]

# Print the results
print("Predicted Sentiment:", predicted_sentiment)
print("Negative Score:", negative_score)
print("Neutral Score:", neutral_score)
print("Positive Score:", positive_score)


1/1 [==============================] - 0s 17ms/step
Predicted Sentiment: negative
Negative Score: 0.998483
Neutral Score: 0.0014760104
Positive Score: 4.1119016e-05


In [125]:
# Save the model
model.save('model.h5')

In [126]:
test_feature = vectorizer.transform(['Movie is good'])
model.predict(test_feature)

1/1 [==============================] - 0s 21ms/step


array([[0.5432925 , 0.01402593, 0.4426815 ]], dtype=float32)

In [127]:
test_feature = vectorizer.transform(['Nice'])
model.predict(test_feature)

1/1 [==============================] - 0s 14ms/step


array([[0.21323696, 0.42080504, 0.36595798]], dtype=float32)

In [118]:
test_feature = vectorizer.transform(['I\'m okay'])
model.predict(test_feature)

1/1 [==============================] - 0s 17ms/step


array([[5.2758912e-03, 9.9468154e-01, 4.2592150e-05]], dtype=float32)

In [128]:
test_feature = vectorizer.transform(['better'])
model.predict(test_feature)

1/1 [==============================] - 0s 16ms/step


array([[0.55579513, 0.02796437, 0.41624048]], dtype=float32)

In [51]:
vectorizer = TfidfVectorizer(max_features=10000)  # You can adjust max_features as needed
train_features = vectorizer.fit_transform(train_x)

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

AttributeError: 'csr_matrix' object has no attribute 'lower'

In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

max_sequence_length = 100  # Example: maximum sequence length after padding

In [86]:
max_sequence_length = train_x.shape[1]  # Adjusting max_sequence_length to match the number of features

In [87]:
from sklearn.preprocessing import LabelEncoder

# Convert string labels to integer labels
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

In [88]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(max_sequence_length,)),  
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [84]:
# Save the model to HDF5 file
model.save('model1.h5')

AttributeError: 'LogisticRegression' object has no attribute 'save'

In [18]:
import pickle

In [19]:
pickl = {'vectorizer': vectorizer,
         'model': model
         }
pickle.dump(pickl, open('models'+".p", "wb"))

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
tokenizer = Tokenizer(num_words=100)

In [143]:
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

In [144]:
import numpy as np

# Convert csr_matrix to array
train_x_array = train_x.toarray()

# Convert array to list of strings
train_texts = [' '.join(map(str, row)) for row in train_x_array]

# Now you can use train_texts to fit the tokenizer
tokenizer.fit_on_texts(train_texts)

In [141]:
import joblib

# Load the pre-fitted model and vectorizer
model = joblib.load('logistic_regression.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Transform new data using the loaded vectorizer
test_feature = vectorizer.transform(['I\'m okay'])

# Predict the sentiment using the loaded model
prediction = model.predict(test_feature)

# Print the predicted sentiment
print("Predicted Sentiment:", prediction)


Predicted Sentiment: ['neutral']


In [142]:
import joblib

# Load the pre-fitted model and vectorizer
model = joblib.load('logistic_regression.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Transform new data using the loaded vectorizer
test_feature = vectorizer.transform(['better'])

# Make predictions using the loaded model
predictions = model.predict(test_feature)
print(predictions)

['negative']


In [140]:
import joblib

# Load the pre-fitted model and vectorizer
model = joblib.load('logistic_regression.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')
test_feature = vectorizer.transform(['Good'])

# Make predictions using the loaded model
predictions = model.predict(test_feature)
print(predictions)

['neutral']


In [138]:
import joblib

# Load the pre-trained model and vectorizer
model = joblib.load('logistic_regression.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Preprocess new data using the loaded vectorizer
new_data = ['Very bad', 'This product is amazing']
new_data_features = vectorizer.transform(new_data)

# Use the preprocessed data to make predictions using the loaded model
predictions = model.predict(new_data_features)

# Print the predictions
for text, prediction in zip(new_data, predictions):
    print(f'Text: {text} - Predicted Sentiment: {prediction}')


Text: Very bad - Predicted Sentiment: negative
Text: This product is amazing - Predicted Sentiment: positive


In [136]:
# Assuming 'x_vectorized' contains the vectorized representation of the entire dataset

# Use the loaded model to make predictions on the entire dataset
predictions = model.predict(x_vectorized)

# Print the predictions
for text, prediction in zip(x, predictions):
    print(f'Text: {text} - Predicted Sentiment: {prediction}')


Text: southwestair understand weather issue time cancelled flighted reinstated cancelled flighted hardly update nothing - Predicted Sentiment: negative
Text: united cool customer told care either expected - Predicted Sentiment: negative
Text: americanair trying desperately get boyfriend booked u airway flight price help - Predicted Sentiment: neutral
Text: americanair flight get cancelled flightled able reschedule date want next day day - Predicted Sentiment: negative
Text: united terrible illogical route cancelled flightled connection made pay check bag lost bag ual - Predicted Sentiment: negative
Text: united people stranded cause suck better yet weasel flight booking problem room people claiming weather http co flcnnn usd - Predicted Sentiment: negative
Text: virginamerica site back - Predicted Sentiment: neutral
Text: united see two flight two seat morning get - Predicted Sentiment: neutral
Text: usairways first pilot late flight maintenance exactly happened two week ago unacceptab

In [74]:
import pandas as pd

# Load the dataset
data = pd.read_csv('Reviews.csv')  # Replace 'your_dataset.csv' with the path to your dataset
# Preprocess the entire dataset using the loaded vectorizer
data_features = vectorizer.transform(data['Text'])

# Use the preprocessed data to make predictions using the loaded model
predictions = model.predict(data_features)

# Add the predictions to the DataFrame
data['predicted_sentiment'] = predictions

# Save the DataFrame with predictions if needed
data.to_csv('data_with_predictions.csv', index=False)  # Replace 'data_with_predictions.csv' with the desired output file path


In [135]:
import numpy as np

# Make predictions on the entire dataset
predicted_probabilities = model.predict_proba(data_features)

# Extract the index of the highest probability for each sample
predicted_labels = np.argmax(predicted_probabilities, axis=1)

# Convert predicted labels to sentiment categories
sentiment_categories = ['negative', 'neutral', 'positive']
predicted_sentiments = [sentiment_categories[label] for label in predicted_labels]

# Add predicted sentiment and scores to the DataFrame
data['predicted_sentiment'] = predicted_sentiments
data['negative_score'] = predicted_probabilities[:, 0]
data['neutral_score'] = predicted_probabilities[:, 1]
data['positive_score'] = predicted_probabilities[:, 2]

# Save the DataFrame with predictions and sentiment scores if needed
data.to_csv('data_with_predictions_and_scores.csv', index=False)  # Replace 'data_with_predictions_and_scores.csv' with the desired output file path


In [131]:
# Transform the text using the vectorizer
test_feature = vectorizer.transform(['Its better than anything'])

# Predict the sentiment
predicted_probabilities = model.predict(test_feature)

# Extract the index of the highest probability
predicted_label = np.argmax(predicted_probabilities)

# Convert predicted label to sentiment category
sentiment_categories = ['negative', 'neutral', 'positive']
predicted_sentiment = sentiment_categories[predicted_label]

# Get the probability for each sentiment
negative_score = predicted_probabilities[0][0]
neutral_score = predicted_probabilities[0][1]
positive_score = predicted_probabilities[0][2]

# Print the results
print("Predicted Sentiment:", predicted_sentiment)
print("Negative Score:", negative_score)
print("Neutral Score:", neutral_score)
print("Positive Score:", positive_score)


1/1 [==============================] - 0s 14ms/step
Predicted Sentiment: negative
Negative Score: 0.9080935
Neutral Score: 0.049750496
Positive Score: 0.04215609


In [132]:
# Transform the text using the vectorizer
test_feature = vectorizer.transform(['Meat Week Day 3: Tummy hurts every night'])

# Predict the sentiment
predicted_probabilities = model.predict(test_feature)

# Extract the index of the highest probability
predicted_label = np.argmax(predicted_probabilities)

# Convert predicted label to sentiment category
sentiment_categories = ['negative', 'neutral', 'positive']
predicted_sentiment = sentiment_categories[predicted_label]

# Get the probability for each sentiment
negative_score = predicted_probabilities[0][0]
neutral_score = predicted_probabilities[0][1]
positive_score = predicted_probabilities[0][2]

# Print the results
print("Predicted Sentiment:", predicted_sentiment)
print("Negative Score:", negative_score)
print("Neutral Score:", neutral_score)
print("Positive Score:", positive_score)


1/1 [==============================] - 0s 17ms/step
Predicted Sentiment: negative
Negative Score: 0.998483
Neutral Score: 0.0014760104
Positive Score: 4.1119016e-05


In [133]:
# Transform the text using the vectorizer
test_feature = vectorizer.transform(['good'])

# Predict the sentiment
predicted_probabilities = model.predict(test_feature)

# Extract the index of the highest probability
predicted_label = np.argmax(predicted_probabilities)

# Convert predicted label to sentiment category
sentiment_categories = ['negative', 'neutral', 'positive']
predicted_sentiment = sentiment_categories[predicted_label]

# Get the probability for each sentiment
negative_score = predicted_probabilities[0][0]
neutral_score = predicted_probabilities[0][1]
positive_score = predicted_probabilities[0][2]

# Print the results
print("Predicted Sentiment:", predicted_sentiment)
print("Negative Score:", negative_score)
print("Neutral Score:", neutral_score)
print("Positive Score:", positive_score)


1/1 [==============================] - 0s 18ms/step
Predicted Sentiment: neutral
Negative Score: 0.03966935
Neutral Score: 0.50468
Positive Score: 0.45565063
